In [2]:
# ERKAS Excel tpo GPKG export
# DF, 2022-03-29. Contact: florian.denzinger@bafu.admin.ch
### old version with ogr2ogr compatibility

def list_file_paths(folder_path, file_type):
    """This function creates a list of specific file paths in a directory and its subdirectories.

    Args:
        folder_path (string): Path to the folder to list files
        file_type (string): Filetype specification of type string (e.g. "*.shp"). If you want to list all files specify

    Returns:
        string: Pathlib object with specific file paths in the folder directory.
    """
    from pathlib import Path
    path = Path(folder_path)
    file_path_list = [f for f in path.rglob(file_type)]
    return file_path_list


def ilixtf2gpkg(ili_dir, gpkg_dir):
    """This function converts ILI XTF file(s) in a directory to GPKG file(s).

    Args:
        ili_dir (string): Path to the directory where ILI XTF files are stored
        gpkg_dir (string): Path to the directory where GPKG files should be stored
    """
    from osgeo import ogr
    import os
    if not os.path.exists(gpkg_dir):
        os.makedirs(gpkg_dir)

    ili_files = list_file_paths(ili_dir, '*.xtf')

    for ili_file in ili_files:
        print(f"Processing: {ili_file.stem}")
        command = ("ogr2ogr -f" + " " + 'GPKG' + " " + str(os.path.join(gpkg_dir, ili_file.stem+".gpkg")) + " " + str(ili_file))
        print(f"Processing: {ili_file.stem}")
        os.system(command, )



import warnings
import os
warnings.filterwarnings("ignore")

# execute the script
workingdir = os.getcwd()

# prepare the ILI XTF files and export to .gpkg
ili_dir = os.path.join(workingdir, 'Data/ILI')
gpkg_dir = os.path.join(workingdir, 'Data/ILI_GPKG')
ilixtf2gpkg(ili_dir, gpkg_dir)

# import gpkg with geopandas
import geopandas as gpd
gpkg_files = list_file_paths(gpkg_dir, '*.gpkg')
for gpkg_file in gpkg_files:
    print(f"Processing: {gpkg_file.stem}")
    gdf = gpd.read_file(gpkg_file)

Processing: ERKAS_Strassen_BL_2021
Processing: ERKAS_Strassen_BL_2021


Warning 6: Failed to add new definition to existing layers, attributes not saved: DatumREPers, 


Processing: ERKAS_Strassen_SO_2021
Processing: ERKAS_Strassen_SO_2021
Processing: ERKAS_Strassen_AG_2021
Processing: ERKAS_Strassen_AG_2021
Processing: ERKAS_Strassen_GR_2021
Processing: ERKAS_Strassen_GR_2021


Warning 6: Failed to add new definition to existing layers, attributes not saved: REPflichtGW, REPflichtOFG, REPflichtPers, 


Processing: ERKAS_Strassen_TG_2021
Processing: ERKAS_Strassen_TG_2021
Processing: CARAM_routes_TI_2021
Processing: CARAM_routes_TI_2021
Processing: Erkas_Strassen_BE_2021
Processing: Erkas_Strassen_BE_2021
Processing: ERKAS_Strassen_SG_2021
Processing: ERKAS_Strassen_SG_2021


Warning 6: Failed to add new definition to existing layers, attributes not saved: SegmentNameGW, SegmentNameOFG, 


Processing: ERKAS_Strassen_LU_2021
Processing: ERKAS_Strassen_LU_2021
Processing: ERKAS_Strassen_AG_2021
Processing: ERKAS_Strassen_SO_2021
Processing: Erkas_Strassen_BE_2021
Processing: ERKAS_Strassen_LU_2021
Processing: ERKAS_Strassen_SG_2021
Processing: ERKAS_Strassen_TG_2021
Processing: ERKAS_Strassen_GR_2021
Processing: ERKAS_Strassen_BL_2021
Processing: CARAM_routes_TI_2021


In [108]:
# import gpkg with geopandas (working)
import geopandas as gpd
import fiona
import re

gpkg_files = list_file_paths(gpkg_dir, '*.gpkg')
for gpkg_file in gpkg_files:
    print(f"Processing: {gpkg_file.stem}")
    gpkg_layers = fiona.listlayers(gpkg_file)
    for gpkg_layer in gpkg_layers:
        if '.Datenpunkt' in gpkg_layer:
            print(f"Processing: {gpkg_layer}")
            gpkg_layer = gpkg_layers[0]
            model_version = re.findall("[V][0-9]\_[0-9]|[V][0-9][.]", gpkg_layer)
            '''coordinate_system_string = re.findall(r"[L][V][0][3]|[L][V][9][5]", gpkg_layer)
            if coordinate_system_string == ["LV95"]:
                crs = "EPSG:2056"
            elif coordinate_system_string == ["LV03"]:
                crs = "EPSG:21781"
            print(f"Coordinate system: {crs}")'''
            print(f"ILI model version: {model_version}")
            gdf = gpd.read_file(gpkg_file)
            '''gdf = gdf.set_crs(crs, allow_override=True)
            if crs == "EPSG:21781":
                gdf = gdf.to_crs("EPSG:2056")
                print(f"Projected")'''
            print(f"Number of columns: {len(gdf.columns)}")
            #print(gdf.crs)
            gdf.to_file(os.path.join(workingdir, 'Data/ILI_SHP/'+gpkg_file.stem+'.shp'), driver='ESRI Shapefile')
            print(f"Number of points in dataset: {len(gdf.length)}")
            print("________________________________________________________________________")

Processing: ERKAS_Strassen_AG_2021
Processing: ERKAS_Strassen_LV95_V1.ERKAS_Strassen.Datenpunkt
ILI model version: ['V1.']
Number of columns: 10
Number of points in dataset: 61475
________________________________________________________________________
Processing: ERKAS_Strassen_SO_2021
Processing: ERKAS_Strassen_LV95_V1.ERKAS_Strassen.Datenpunkt
ILI model version: ['V1.']
Number of columns: 10
Number of points in dataset: 25828
________________________________________________________________________
Processing: Erkas_Strassen_BE_2021
Processing: ERKAS_Strassen_LV95_V1.ERKAS_Strassen.Datenpunkt
ILI model version: ['V1.']
Number of columns: 10
Number of points in dataset: 133369
________________________________________________________________________
Processing: ERKAS_Strassen_LU_2021
Processing: ERKAS_Strassen_Kataster_LV95_V2_0.ERKAS_Strassen_Kataster.Datenpunkt
ILI model version: ['V2_0']
Number of columns: 11
Number of points in dataset: 36618
_______________________________________

In [680]:
def filter_df_column_names(gdf_column_list, filter):
    """Filter column names of a dataframe.

    Args:
        gdf_column_list (list): List of column names of a dataframe
        filter (string): String that partially contains the name of the column_
    """
    filter_string = [string for string in gdf_column_list if filter in string]
    return(filter_string)

def unnest_list(filter_list):
    """Unnests a list of lists.

    Args:
        filter_list (list): List of lists to unnest
    """
    from itertools import chain
    unnested = list(chain(*filter_list))
    return(unnested)

def calculate_statistics_from_ili_gpkg(gpkg_dir, excel_export_path):
    """Calculates statistics for ILI converted GPKG file (compatible with ERKAS Strassen >V2_0)

    Args:
        gpkg_dir (string): Path to the folder location of the converted ILI_GPKG files (need to be in GPKG format)
        excel_export_path (string): Path where the excel file containing the statistics is saved to
    """
    import pandas as pd

    gpkg_files = list_file_paths(gpkg_dir, '*.gpkg')

    df_results = pd.DataFrame()
    for gpkg_file in gpkg_files:
        print(f"Processing: {gpkg_file.stem}")
        
        gdf = gpd.read_file(gpkg_file, crs="EPSG:2056")
        gdf = gdf.set_crs("EPSG:2056", allow_override=True)
        print(f"Number of points: {len(gdf.length)}")
        print(f"Coordinate system: {gdf.crs}")
        print(f"Number of columns (intial dataset): {len(gdf.columns)}")

        idlaenge = filter_df_column_names(list(gdf.columns), "IDLaenge")
        kbfrei = filter_df_column_names(list(gdf.columns), "KBfrei")
        ampelcodepers = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodePers")
        ampelcodeofg = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodeOFG")
        ampelcodegw = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodeGW")

        filter_list = idlaenge, kbfrei, ampelcodepers, ampelcodeofg, ampelcodegw
        filter = unnest_list(filter_list)
        filter.append('geometry')
        canton_name_short = gpkg_file.stem.split('_')[-2]
        
        gdf_subset = gdf[filter]
        gdf_subset['Inhaber'] = canton_name_short
        print(f"Columns of subset: {len(gdf_subset.columns)}")
        if len(gdf_subset.columns) != 7:
            print("___________________________________________________")
            continue
        if not os.path.exists(workingdir + '/Data/ILI_GPKG_EXPORT/'):
            os.makedirs(os.path.join(workingdir + '/Data/ILI_GPKG_EXPORT/'))
        gdf_subset.to_file(os.path.join(workingdir, 'Data/ILI_GPKG_EXPORT/'+gpkg_file.stem+'.gpkg'), driver='GPKG')

        # calculate total of kilometer Durchgangsstrasse
        gdf_subset.IDLaenge = pd.to_numeric(gdf_subset.IDLaenge)
        total = gdf_subset.IDLaenge.sum()/1000
        print(f"Total Anzahl Kilometer Durchgangsstrasse: {total}")
        # calculate total KB befreit
        kbfrei_grouped = gdf_subset.groupby(kbfrei)[idlaenge].sum().reset_index()
        kb_befreit = kbfrei_grouped[kbfrei_grouped[''.join(kbfrei)]=='true']
        kbfrei_grouped[kbfrei_grouped[''.join(kbfrei)]=='true']
        if kb_befreit.empty:
            kb_befreit_calc = 0
        else:
            kb_befreit_calc = kb_befreit.values()
        print(f"KB befreit: {kb_befreit_calc}")
        # calculate Ampelcodes
        ampelcodepers_grouped = gdf_subset.groupby(ampelcodepers)[idlaenge].sum().reset_index()
        ampelcodepers_calc = unnest_list(ampelcodepers_grouped[idlaenge].values/1000)
        ampelcodepers_calc_classes = unnest_list(ampelcodepers_grouped[ampelcodepers].astype(int).values)
        print(f"Ampelcode Pers Classes: {ampelcodepers_calc_classes}") 
        print(f"Ampelcode Pers: {ampelcodepers_calc}")
        ampelcodeofg_grouped = gdf_subset.groupby(ampelcodeofg)[idlaenge].sum().reset_index()
        ampelcodeofg_calc = unnest_list(ampelcodeofg_grouped[idlaenge].values/1000)
        ampelcodeofg_calc_classes = unnest_list(ampelcodeofg_grouped[ampelcodeofg].astype(int).values)
        print(f"Ampelcode OFG Classes: {ampelcodeofg_calc_classes}") 
        print(f"Ampelcode OFG: {ampelcodeofg_calc}")
        ampelcodegw_grouped = gdf_subset.groupby(ampelcodegw)[idlaenge].sum().reset_index()
        ampelcodegw_calc = unnest_list(ampelcodegw_grouped[idlaenge].values/1000)
        ampelcodegw_calc_classes = unnest_list(ampelcodegw_grouped[ampelcodegw].astype(int).values)
        print(f"Ampelcode GW Classes: {ampelcodegw_calc_classes}")
        print(f"Ampelcode GW: {ampelcodegw_calc}")
        ampelcodepers_dict = {ampelcodepers_calc_classes[i]: ampelcodepers_calc[i] for i in range(len(ampelcodepers_calc_classes))}
        ampelcodepers_dict = {f"AmpelcodePers{key}": val for key, val in ampelcodepers_dict.items()}
        ampelcodeofg_dict = {ampelcodeofg_calc_classes[i]: ampelcodeofg_calc[i] for i in range(len(ampelcodeofg_calc_classes))}
        ampelcodeofg_dict = {f"AmpelcodeOFG{key}": val for key, val in ampelcodeofg_dict.items()}
        ampelcodegw_dict = {ampelcodegw_calc_classes[i]: ampelcodegw_calc[i] for i in range(len(ampelcodegw_calc_classes))}
        ampelcodegw_dict = {f"AmpelcodeGW{key}": val for key, val in ampelcodegw_dict.items()}
        df = pd.concat([pd.DataFrame([canton_name_short], columns=['Inhaber']), pd.DataFrame([total], columns=['total_km_durchgangsstrasse']), pd.DataFrame([kb_befreit_calc], columns=['kb_befreit'])], axis=1)
        df = pd.concat([df, pd.DataFrame([ampelcodepers_dict])], axis=1)
        df = pd.concat([df, pd.DataFrame([ampelcodeofg_dict])], axis=1)
        df = pd.concat([df, pd.DataFrame([ampelcodegw_dict])], axis=1)
        df_results = pd.concat([df_results,df],axis=0)
        print("___________________________________________________")

    df_results
    df_results.to_excel(excel_export_path, index=False)


gpkg_dir = r'/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG_CONVERT'
calculate_statistics_from_ili_gpkg(gpkg_dir, r"/Users/floriandenzinger/Code/ERKAS-processing/Data/RESULTS/ILI_GPKG_STATISTICS.xlsx")


Processing: ERKAS_Strassen_AG_2021
Number of points: 61475
Coordinate system: EPSG:2056
Number of columns (intial dataset): 31
Columns of subset: 6
___________________________________________________
Processing: ERKAS_Strassen_TI_2021
Number of points: 5963
Coordinate system: EPSG:2056
Number of columns (intial dataset): 31
Columns of subset: 6
___________________________________________________
Processing: ERKAS_Strassen_SO_2021
Number of points: 25828
Coordinate system: EPSG:2056
Number of columns (intial dataset): 31
Columns of subset: 6
___________________________________________________
Processing: ERKAS_Strassen_BE_2021
Number of points: 133369
Coordinate system: EPSG:2056
Number of columns (intial dataset): 23
Columns of subset: 6
___________________________________________________
Processing: ERKAS_Strassen_LU_2021
Number of points: 36618
Coordinate system: EPSG:2056
Number of columns (intial dataset): 32
Columns of subset: 7
Total Anzahl Kilometer Durchgangsstrasse: 365.898
KB

In [256]:

gpkg_file = gpkg_files[0]
print(f"Processing: {gpkg_file.stem}")
gdf = gpd.read_file(gpkg_file, crs="EPSG:2056")
gdf = gdf.set_crs("EPSG:2056", allow_override=True)
print(f"Number of points: {len(gdf.length)}")
print(f"Coordinate system: {gdf.crs}")
print(f"Number of columns: {len(gdf.columns)}")

idlaenge = filter_df_column_names(list(gdf.columns), "IDLaenge")
ampelcodepers = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodePers")
ampelcodeofg = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodeOFG")
ampelcodegw = filter_df_column_names(list(gdf.columns), "Ergebnis_AmpelCodeGW")

filter_list = idlaenge, ampelcodepers, ampelcodeofg, ampelcodegw
filter = unnest_list(filter_list)
filter.append('geometry')

canton_name_short = gpkg_file.stem.split('_')[-2]
gdf_subset = gdf[filter]
gdf_subset['Inhaber'] = canton_name_short
#gdf_subset.to_file(os.path.join(workingdir, 'Data/ILI_SUBSET_GPKG/'+gpkg_file.stem+'.gpkg'), driver='GPKG')
print(f"Columns of subset: {len(gdf_subset.columns)}")
print("_________________")



Processing: ERKAS_Strassen_AG_2021
Number of points: 61475
Coordinate system: EPSG:2056
Number of columns: 31
Columns of subset: 5
_________________


In [245]:
# calculate distance
gdf_distance = gpd.GeoDataFrame(geometry=gpd.points_from_xy(gdf_subset.geometry.x, gdf_subset.geometry.y), crs="EPSG:2056")
gdf_distance['IDLaenge'] = gdf_distance.distance(gdf_distance.shift(1))
gdf_distance['IDLaenge'] = gdf_distance['IDLaenge']/1000
df  = gdf_subset.merge(gdf_distance, on='geometry', how='left')
gdf_subset = gpd.GeoDataFrame(df, crs="EPSG:2056")

In [57]:
# regex model_version

import re
import fiona
for gpkg_file in gpkg_files:
    print(gpkg_file)
    gpkg_layers = fiona.listlayers(gpkg_file)
    gpkg_layer = gpkg_layers[0]
    model_version = re.findall("[V][0-9]\_[0-9]|[V][0-9][.]", gpkg_layer)
    print(model_version)
    print("______________")

/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_AG_2021.gpkg
['V1.']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_SO_2021.gpkg
['V1.']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/Erkas_Strassen_BE_2021.gpkg
['V1.']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_LU_2021.gpkg
['V2_0']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_SG_2021.gpkg
['V1_1']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_TG_2021.gpkg
['V2_0']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_GR_2021.gpkg
['V1.']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/ERKAS_Strassen_BL_2021.gpkg
['V2_0']
______________
/Users/floriandenzinger/Code/ERKAS-processing/Data/ILI_GPKG/CARAM_routes_TI_2021.gpkg
['V1.']
______________

In [59]:

# prepare the ILI XTF files and export to .gpkg
ili_dir = os.path.join(workingdir, 'Data/ILI')
gpkg_dir = os.path.join(workingdir, 'Data/ILI_GPKG')
ilixtf2gpkg(ili_dir, gpkg_dir)'''

'''# import gpkg with geopandas
import geopandas as gpd
import fiona
import re

gpkg_files = list_file_paths(gpkg_dir, '*.gpkg')
for gpkg_file in gpkg_files:
    print(f"Processing: {gpkg_file.stem}")
    gpkg_layers = fiona.listlayers(gpkg_file)
    for gpkg_layer in gpkg_layers:
        if '.Datenpunkt' in gpkg_layer:
            print(gpkg_layer)
            gpkg_layer = gpkg_layers[0]
            model_version = re.findall("[V][0-9]\_[0-9]|[V][0-9][.]", gpkg_layer)
            coordinate_system_string = re.findall(r"[L][V][0][3]|[L][V][9][5]", gpkg_layer)
            print(coordinate_system_string)
            if coordinate_system_string == ["LV95"]:
                crs = "EPSG:2056"
            elif coordinate_system_string == ["LV03"]:
                crs = "EPSG:21781"
            print(f"Coordinate system: {crs}")
            print(f"ILI model version: {model_version}")
            gdf = gpd.read_file(gpkg_file)
            gdf = gdf.set_crs(crs, allow_override=True)
            print(f"Number of columns: {len(gdf.columns)}")
            print(gdf.crs)
            print("________________________________________________________________________")


















erkas_csv_filepaths = list_file_paths(r"O:\GIS\GEP\RLS\_Mitarbeitende\DF2020\ERKAS\01_csv_bereinigt", "*.csv")

for erkas_csv_filepath in erkas_csv_filepaths:
    print(erkas_csv_filepath)
    df = pd.read_csv(erkas_csv_filepath, skiprows=2)
    header = df.iloc[0]
    df.columns = header
    df = df.drop(df.index[[0,1,2]])
    df.reset_index(drop=True, inplace=True)
    df = df.drop(df.columns[[0]], axis=1)

    # set value '<Nul>' to Nodata
    df = df.replace('<Nul>', np.NaN)

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.loc[:, 'Ort_E-Coord'], df.loc[:,'Ort_N-Coord']))
    gdf = gdf.set_crs('epsg:2056')
    gdf.to_file(os.path.join(r"O:\GIS\GEP\RLS\_Mitarbeitende\DF2020\ERKAS\02_csv_gpkg_export",
                             erkas_csv_filepath.stem +".gpkg"), driver = 'GPKG')

erkas_csvgpkgs_filepaths = list_file_paths(r'O:\GIS\GEP\RLS\_Mitarbeitende\DF2020\ERKAS Verkehrswege 2021_Bereinigt_DF\02_csv_gpkg_export', "*.gpkg")

for erkas_csvgpkgs_filepath in erkas_csvgpkgs_filepaths:
    #print(erkas_csvgpkgs_filepath)
    df = gpd.read_file(erkas_csvgpkgs_filepath)
    #df = df.fillna(9999)
    canton_name_short = erkas_csvgpkgs_filepath.name.split('_')[2]
    print("Canton:", canton_name_short)
    # convert column IDLaenge to int
    df['IDLaenge'] = pd.to_numeric(df['IDLaenge'])
    # get total sum of Strassenkilometer
    total_sum_strassenkilometer = df['IDLaenge'].sum()/1000
    print("Strassenkilometer:", total_sum_strassenkilometer)
    # replace
    df['KBfrei'] = df['KBfrei'].replace('false', 0)
    df['KBfrei'] = df['KBfrei'].replace('true', 1)
    df['KBfrei'] = df['KBfrei'].replace('FALSE', 0)
    df['KBfrei'] = df['KBfrei'].replace('TRUE', 1)
    df['KBfrei'] = df['KBfrei'].replace('N', 0)
    df['KBfrei'] = df['KBfrei'].replace('0', 0)
    df['KBfrei'] = df['KBfrei'].replace('1', 1)
    sum_KBfrei = df.groupby(['KBfrei'])['IDLaenge'].sum().reset_index()
    if not sum_KBfrei[sum_KBfrei['KBfrei'] == 1].values.tolist() == []:
        sum_KBfrei = sum_KBfrei[sum_KBfrei['KBfrei']==1].values.tolist()[0][1]
    else:
        sum_KBfrei = 0
    sum_KBfrei = sum_KBfrei/1000
    print("KB befreit:", sum_KBfrei)
    # get total sum of AmpelCode classes
    sum_ampel_codes = df.groupby(['AmpelCodePers'])['IDLaenge'].sum().reset_index()
    sum_ampel_codes = sum_ampel_codes['IDLaenge'].values.tolist()
    print("Ampel Codes:", sum_ampel_codes)
    row = canton_name_short, total_sum_strassenkilometer, sum_KBfrei , *sum_ampel_codes

erkas_csvgpkgs_filepaths = list_file_paths(r'O:\GIS\GEP\RLS\_Mitarbeitende\DF2020\ERKAS Verkehrswege 2021_Bereinigt_DF\02_xtf_gpkg_export', "*.gpkg")

for erkas_csvgpkgs_filepath in erkas_csvgpkgs_filepaths:
    #print(erkas_csvgpkgs_filepath)
    df = gpd.read_file(erkas_csvgpkgs_filepath)
    #df = df.fillna(9999)
    canton_name_short = erkas_csvgpkgs_filepath.name.split('_')[2]
    print("Canton:", canton_name_short)
    # convert column IDLaenge to int
    df['IDLaenge'] = pd.to_numeric(df['IDLaenge'])
    # get total sum of Strassenkilometer
    total_sum_strassenkilometer = df['IDLaenge'].sum()/1000
    print("Strassenkilometer:", total_sum_strassenkilometer)
    # replace
    df['KBfrei'] = df['KBfrei'].replace('false', 0)
    df['KBfrei'] = df['KBfrei'].replace('true', 1)
    df['KBfrei'] = df['KBfrei'].replace('FALSE', 0)
    df['KBfrei'] = df['KBfrei'].replace('TRUE', 1)
    df['KBfrei'] = df['KBfrei'].replace('N', 0)
    df['KBfrei'] = df['KBfrei'].replace('0', 0)
    df['KBfrei'] = df['KBfrei'].replace('1', 1)
    sum_KBfrei = df.groupby(['KBfrei'])['IDLaenge'].sum().reset_index()
    if not sum_KBfrei[sum_KBfrei['KBfrei'] == 1].values.tolist() == []:
        sum_KBfrei = sum_KBfrei[sum_KBfrei['KBfrei']==1].values.tolist()[0][1]
    else:
        sum_KBfrei = 0
    sum_KBfrei = sum_KBfrei/1000
    print("KB befreit:", sum_KBfrei)
    # get total sum of AmpelCode classes
    sum_ampel_codes = df.groupby(['AmpelCodePers'])['IDLaenge'].sum().reset_index()
    sum_ampel_codes = sum_ampel_codes['IDLaenge'].values.tolist()
    print("Ampel Codes:", sum_ampel_codes)
    row = canton_name_short, total_sum_strassenkilometer, sum_KBfrei , *sum_ampel_codes


,TID,ID,Inhaber,StrassenNr,Streckentyp,Streckenbezeichnung,rErgebnis,rVerkehrsaufkommen,rVollzug,geometry
0,,1358205,TI,N2_ABELN,O,,ER.1358205,VE.1358205,VO.1358205,POINT (2723861.96300 1120259.33700)
1,,1364475,TI,PA399.1,O,Allacciamento Vezia - Massagno,ER.1364475,VE.1364475,VO.1364475,POINT (2715764.06500 1098206.79900)
2,,1360863,TI,P2,O,Motto Bartola-Airolo-Chiasso,ER.1360863,VE.1360863,VO.1360863,POINT (2714513.01600 1109771.45300)
3,,1369488,TI,P2,O,Motto Bartola-Airolo-Chiasso,ER.1369488,VE.1369488,VO.1369488,POINT (2714519.75900 1109764.06800)
4,,1364472,TI,PA399.1,O,Allacciamento Vezia - Massagno,ER.1364472,VE.1364472,VO.1364472,POINT (2715759.98900 1098215.93100)
...,...,...,...,...,...,...,...,...,...,...
5958,,1362807,TI,P13,O,Lumino-Bellinzona-Ascona,ER.1362807,VE.1362807,VO.1362807,POINT (2705237.08200 1113948.71800)
5959,,1362810,TI,P13,O,Lumino-Bellinzona-Ascona,ER.1362810,VE.1362810,VO.1362810,POINT (2705241.78800 1113939.89400)
5960,,1373754,TI,PA398,O,Fornasette-Pte Tresa-Agno-Lamone,ER.1373754,VE.1373754,VO.1373754,POINT (2714260.92200 1097413.32800)
5961,,1357077,TI,P404,O,Paradiso-Grancia-Figino,ER.1357077,VE.1357077,VO.1357077,POINT (2715477.83100 1092621.04700)
